roadway network builder
description: create roadway features after centroid connector builder
Sijia.Wang@wsp.com
Feb 12, 2019

In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
from shapely.geometry import Point, shape, LineString

In [17]:
#define input files
#osm drive network (edge/node), centroid connector file, node coordinates
cd = "C:/Users/wangs1/Documents/Met_Council/"
osm_edge_file = cd + "centroid_connector_builder/osm_edges_with_cc.shp"
county_shape_file = cd + "GIS/cb_2017_us_county_5m/cb_2017_us_county_5m.shp"

#define county name codes
county_name_dict = {'Anoka':1, 'Carver':2, 'Dakota':3, 'Hennepin':4, 'Ramsey':5, 'Scott':6, 'Washington':7, 'external':10,
                    'Chisago':11, 'Goodhue':12, 'Isanti':13, 'Le Sueur':14, 'McLeod':15, 'Pierce':16, 'Polk':17, 'Rice':18,
                    'Sherburne':19, 'Sibley':20, 'St. Croix':21, 'Wright':22}

In [18]:
osm_edge_gdf = gpd.read_file(osm_edge_file)
osm_edge_gdf = osm_edge_gdf.to_crs({'init': 'epsg:26915', 'no_defs': True})
county_shape_gdf = gpd.read_file(county_shape_file)
county_shape_gdf = county_shape_gdf.to_crs(epsg = 26915)

In [19]:
# add county
#osm_edge_gdf['ID'] = range(1, 1+len(osm_edge_gdf))
osm_edge_gdf['mid_point'] = osm_edge_gdf['geometry'].interpolate(0.5, normalized = True)
osm_edge_gdf.rename(columns = {'mid_point' : 'geometry', 'geometry' : 'line_geom'}, inplace = True)

osm_edge_gdf = gpd.sjoin(osm_edge_gdf, 
                        county_shape_gdf, 
                        how = 'left', 
                        op = 'intersects')

osm_edge_gdf.rename(columns = {'geometry' : 'mid_point', 'line_geom' : 'geometry'}, inplace = True)

osm_edge_gdf['COUNTY'] = osm_edge_gdf['NAME_right'].map(county_name_dict).fillna(10).astype(int)

In [36]:
#add link distance in miles
osm_edge_gdf['DISTANCE'] = osm_edge_gdf['geometry'].length/1609.344
osm_edge_gdf['DISTANCE'] = osm_edge_gdf['DISTANCE'].astype(np.float16)

In [37]:
#Other Attributes
osm_edge_gdf['T_PRIORITY'] = np.float16(0)
osm_edge_gdf['T_MANTIME'] = np.float16(0)
osm_edge_gdf['BIKE'] = 0
osm_edge_gdf['AREA'] = 0
osm_edge_gdf['HOV'] = 0
osm_edge_gdf['AADT'] = 0
osm_edge_gdf['AM_CNT'] = 0
osm_edge_gdf['MD_CNT'] = 0
osm_edge_gdf['PM_CNT'] = 0
osm_edge_gdf['NT_CNT'] = 0
osm_edge_gdf['DY_CNT'] = 0
osm_edge_gdf['ASGNGRP'] = 0
#osm_edge_gdf.rename(columns = {'LANES' :'osm_lanes', 'LANES_CONS' : 'LANES'}, inplace = True)
osm_edge_gdf['LANES'] = osm_edge_gdf['LANES'].astype(int)
osm_edge_gdf['RC_NUM'] = 0

In [40]:
column_list = ['ID', 'osmid_from', 'osmid_to', 'COUNTY', 'T_PRIORITY', 'T_MANTIME', 'DISTANCE', 'BIKE', 'AREA',
       'HOV', 'AADT', 'AM_CNT', 'MD_CNT', 'PM_CNT', 'NT_CNT', 'DY_CNT',
       'ASGNGRP', 'LANES', 'CENTROID', 'RC_NUM', 'geometry']
osm_edge_gdf[column_list].to_file(cd + "/osm_roadway_network_builder/osm_roadway_for_modeling.shp")

In [41]:
pd.DataFrame(osm_edge_gdf[column_list]).rename(columns = {'ID' :'id'}).to_csv(cd + "osm_roadway_network_builder/osm_roadway_for_modeling.csv",
                                              index = False)